In [1]:
import re
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [2]:
train_data = pd.read_csv('/data/ephemeral/code/level2-nlp-datacentric-nlp-05/PMJ/정미노이즈만_제거및증강.csv')
test_data = pd.read_csv("/data/ephemeral/data/train_noised(x)_remove.csv")

In [3]:
import re

def remove_unwanted_characters(text):
    return re.sub(r'[^가-힣a-zA-Z0-9\s\u4e00-\u9fff\.\·]', '', text)

test_data['text'] = test_data['text'].apply(remove_unwanted_characters)

In [4]:
import evaluate
import numpy as np

f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')


In [5]:
data2 = pd.read_csv('/data/ephemeral/data/aug_data_특수문자제거.csv')

train_data = pd.concat([train_data,data2], ignore_index=True)

In [6]:
import wandb
import dataset
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split

model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(train_data['target'].unique()))

dataset_train, dataset_valid = train_test_split(train_data, test_size=0.1, random_state=42)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=128, truncation=True)

train_dataset = Dataset.from_pandas(dataset_train)
test_dataset = Dataset.from_pandas(dataset_valid)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("target", "labels")
test_dataset = test_dataset.rename_column("target", "labels")
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

training_args = TrainingArguments(
    do_train=True,
    do_eval=True,
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=336,
    per_device_eval_batch_size=32,
    logging_strategy="steps",    
    logging_steps=10,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="wandb",             
    run_name="bert-finetuning",
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    lr_scheduler_type='linear',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
wandb.finish()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/10271 [00:00<?, ? examples/s]

Map:   0%|          | 0/1142 [00:00<?, ? examples/s]

/data/ephemeral/code/Mujae/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: phs514525 (phs514525-skhu-ac-kr). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,F1
1,1.244300,1.188379,0.619765
2,1.017900,1.030453,0.660366
3,0.826800,0.881858,0.715544
4,0.640000,0.774029,0.748117
5,0.509100,0.674660,0.785090
6,0.386300,0.599221,0.818428
7,0.321200,0.548193,0.831645
8,0.271100,0.509701,0.845000
9,0.228500,0.498839,0.851659
10,0.194400,0.486991,0.860755


eval/f1,▁▂▄▅▆▇▇███
eval/loss,█▆▅▄▃▂▂▁▁▁
eval/runtime,▆▄█▄▄▆▄▄▃▁
eval/samples_per_second,▃▅▁▅▅▃▅▅▆█
eval/steps_per_second,▃▅▁▅▅▃▅▅▆█
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
train/grad_norm,▁▂▂▄▃▂▂▆▃▂▃▅▂▅█▅▅▄▃▅▃▅▄▃▂▄▄▄▁▂▆
train/learning_rate,███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁▁
train/loss,█▇▅▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
eval/f1,0.86075


In [77]:
data123_ids = data123['ID'].tolist()

# test_data에서 data123의 ID와 겹치는 항목 제거
test_data_filtered = test_data[~test_data['ID'].isin(data123_ids)]

In [7]:
test_data['target'].unique()

array([5, 0, 1, 4, 6, 2, 3])

In [8]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from cleanlab.filter import find_label_issues

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_dataset = Dataset.from_pandas(test_data)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.rename_column("target", "labels")
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataloader = DataLoader(test_dataset, batch_size=8)

pred_probs = []
for batch in tqdm(test_dataloader):
    # 입력 데이터를 GPU로 이동
    batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
    
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()  # CPU로 변환하여 numpy 배열로 저장
    pred_probs.extend(probs)  # 각 배치의 예측 확률을 리스트에 추가

Map:   0%|          | 0/1286 [00:00<?, ? examples/s]

100%|██████████| 161/161 [00:02<00:00, 54.38it/s]


# Cleanlap 구현

In [9]:
class_thresholds = [0, 0, 0, 0, 0, 0, 0]
class_counts = [0,0,0,0,0,0,0]
min_t=100000
max_t=0
for i, t in enumerate(test_data['target']):
    if i<10:
        print(t)
    class_thresholds[t]+=pred_probs[i][t]
    min_t = min(min_t, pred_probs[i][t])
    max_t = max(max_t, pred_probs[i][t])
    class_counts[t]+=1 

5
0
1
4
6
4
1
2
0
3


In [11]:
class_counts

[188, 192, 169, 169, 192, 198, 178]

In [10]:
for i in range(7):
    class_thresholds[i]/=class_counts[i]
class_thresholds

[0.21905294278861975,
 0.2325729634618862,
 0.19151619495324565,
 0.27719528029621826,
 0.2360114565293164,
 0.217135728546918,
 0.198289221914922]

In [12]:
test_data['prob']=None

In [86]:
np.argmax(pred_probs[4])

5

In [13]:
import numpy as np

for i, row in test_data.iterrows():
    target = row['target']
    ID = row['ID']
    test_data['prob'][i] = pred_probs[i][target]

    if pred_probs[i][target]<class_thresholds[target]: #pred_prob이 임계치보다 작으면 ㄱㄱ임
        test_data['target'][i] = np.argmax(pred_probs[i])
        test_data['prob'][i] = max(pred_probs[i])
        
'''
faulted_data = pd.DataFrame(columns=test_data_filtered.columns)

indexes_to_remove = []

for index, row in test_data_filtered.iterrows():
    target = row['target']
    pred_prob = pred_probs[index][target]
    
    # 임계치보다 작으면 faulted_data에 추가
    if pred_prob < class_thresholds[target]:
        faulted_data = pd.concat([faulted_data, pd.DataFrame([row])], ignore_index=True)
        indexes_to_remove.append(index)


test_data_filtered = test_data_filtered.drop(indexes_to_remove).reset_index(drop=True)
'''

"\nfaulted_data = pd.DataFrame(columns=test_data_filtered.columns)\n\nindexes_to_remove = []\n\nfor index, row in test_data_filtered.iterrows():\n    target = row['target']\n    pred_prob = pred_probs[index][target]\n    \n    # 임계치보다 작으면 faulted_data에 추가\n    if pred_prob < class_thresholds[target]:\n        faulted_data = pd.concat([faulted_data, pd.DataFrame([row])], ignore_index=True)\n        indexes_to_remove.append(index)\n\n\ntest_data_filtered = test_data_filtered.drop(indexes_to_remove).reset_index(drop=True)\n"

In [14]:
test_data.to_csv('새로운리라벨링.csv')

In [92]:
data = pd.concat([train_data, test_data_filtered], ignore_index=True)

In [93]:
data.to_csv('노이즈제거증강과team_lucid리라벨.csv')

넘는 것들만 남기는 것들 재학습  
못 넘은 얘들을 추후에 추론해서 리라벨링

In [89]:
class_thresholds

[0.209798470410351,
 0.1830067693485079,
 0.12959277517979428,
 0.27333744410669536,
 0.15849032434946891,
 0.11534278840800823,
 0.17060725676182278]

0: 생활문화
1: 스포츠
2: 정치
3: 사회
4: IT과학
5: 경제
6: 세계

# 예측한걸로 바꾼 test세트까지 넣어서 다시 훈련 3epoch만

In [14]:
train_dataset = Dataset.from_pandas(test_data_filtered)
test_dataset = Dataset.from_pandas(faulted_data)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("target", "labels")
test_dataset = test_dataset.rename_column("target", "labels")
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

training_args = TrainingArguments(
    do_train=True,
    do_eval=True,
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=32,
    logging_strategy="steps",       # 로그를 기록할 빈도 설정
    logging_steps=10,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="wandb",              # wandb에 로그 기록
    run_name="bert-finetuning",
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    lr_scheduler_type='linear',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()
wandb.finish()

Map:   0%|          | 0/241 [00:00<?, ? examples/s]

Map:   0%|          | 0/854 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,F1
1,No log,3.794831,0.008569
2,No log,3.817344,0.010086
3,0.730200,3.852857,0.010387


eval/f1,▁▇█
eval/loss,▁▄█
eval/runtime,▁█▄
eval/samples_per_second,█▁▅
eval/steps_per_second,█▁▅
train/epoch,▁▅▆██
train/global_step,▁▅▆██
train/grad_norm,▁
train/learning_rate,▁
train/loss,▁
eval/f1,0.01039


In [16]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from cleanlab.filter import find_label_issues

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_dataset = Dataset.from_pandas(faulted_data)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.rename_column("target", "labels")
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataloader = DataLoader(test_dataset, batch_size=8)

pred_probs = []
for batch in tqdm(test_dataloader):
    # 입력 데이터를 GPU로 이동
    batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
    
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()  # CPU로 변환하여 numpy 배열로 저장
    pred_probs.extend(probs)  # 각 배치의 예측 확률을 리스트에 추가

Map:   0%|          | 0/854 [00:00<?, ? examples/s]

100%|██████████| 107/107 [00:02<00:00, 52.42it/s]


In [17]:
class_thresholds = [0, 0, 0, 0, 0, 0, 0]
class_counts = [0,0,0,0,0,0,0]
min_t=100000
max_t=0
for i, t in enumerate(faulted_data['target']):
    class_thresholds[t]+=pred_probs[i][t]
    min_t = min(min_t, pred_probs[i][t])
    max_t = max(max_t, pred_probs[i][t])
    class_counts[t]+=1

In [18]:
for i in range(7):
    class_thresholds[i] = class_thresholds[i]/class_counts[i]
class_thresholds

[0.04239409748234053,
 0.015063922255873108,
 0.023420495349589898,
 0.04261608558366648,
 0.05154721043072641,
 0.042103677193102026,
 0.046128244044666644]

In [23]:
faulted_data['text'][2]

'페이스북 인터넷 드론 아퀼라 실물 첫 시험비행 성공'

In [24]:
pred_probs

[array([0.04810762, 0.01225736, 0.01736601, 0.07079424, 0.06350983,
        0.08055759, 0.7074074 ], dtype=float32),
 array([0.02402593, 0.0237755 , 0.01069875, 0.0205342 , 0.18056159,
        0.71953785, 0.02086619], dtype=float32),
 array([0.07058666, 0.02259084, 0.01170012, 0.01605273, 0.7672407 ,
        0.0546742 , 0.05715464], dtype=float32),
 array([0.7728791 , 0.01290491, 0.00988823, 0.09574831, 0.06214085,
        0.02086302, 0.02557554], dtype=float32),
 array([0.02171645, 0.01375254, 0.02513433, 0.16164395, 0.04370159,
        0.5926273 , 0.14142382], dtype=float32),
 array([0.03006359, 0.0095513 , 0.03865003, 0.04369174, 0.03942302,
        0.0592752 , 0.77934515], dtype=float32),
 array([0.13248211, 0.00598491, 0.02683132, 0.7663239 , 0.03802504,
        0.01942836, 0.01092432], dtype=float32),
 array([0.8144394 , 0.01155218, 0.01095938, 0.04442446, 0.08330149,
        0.00973447, 0.02558861], dtype=float32),
 array([0.03110916, 0.8930163 , 0.00916008, 0.00753896, 0.035338

In [27]:
for i, t in enumerate(faulted_data['target']):
    if pred_probs[i][t]<class_thresholds[t]: #pred_prob이 임계치보다 작으면 ㄱㄱ임
        faulted_data['target'][i] = np.argmax(np.array(pred_probs[i]))


In [28]:
faulted_data['target'].value_counts()

target
6    144
1    138
0    127
2    119
4    112
3    108
5    106
Name: count, dtype: int64

In [29]:
faulted_data.to_csv('snu일렉트라라벨링한거학습후고친걸로한번더학습후나머지다시고침.csv',index=False, encoding='utf-8-sig')

In [31]:
combined_data = pd.concat([train_data, faulted_data], ignore_index=True)

combined_data.to_csv('combined_data_snu일렉트라리라벨링.csv', index=False, encoding='utf-8-sig')